# Bronze Layer (Read)

### Bronze → Silver Transformation
#### Purpose: Clean raw sales data for analytics use

In [ ]:
# Import PySpark functions

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when, to_date


spark = SparkSession.builder.getOrCreate()


# Read Bronze layer 

storage_account = dbutils.secrets.get(scope="my_scope", key="storage_account")
bronze_container = "bronze"  # safe to keep
file_name = "supermarket_sales_raw.csv"

bronze_path = f"abfss://{bronze_container}@{storage_account}.dfs.core.windows.net/{file_name}"

df_bronze = spark.read.option("header", True).option("inferSchema", True).csv(bronze_path)


# Silver Layer Transformations

In [ ]:
# Normalize date formats to yyyy-MM-dd

df_clean = df_bronze.withColumn(
    "Order Date",
    when(col("Order Date").rlike(r"^\d{1,2}/\d{1,2}/\d{4}$"), to_date(col("Order Date"), "M/d/yyyy"))
    .when(col("Order Date").rlike(r"^\d{2}-\d{2}-\d{4}$"), to_date(col("Order Date"), "dd-MM-yyyy"))
    .otherwise(None)
)

In [ ]:
# Drop rows with missing critical data

df_clean = df_clean.dropna(subset=["Order ID", "Order Date", "Sales"])

In [ ]:
# Define Silver path
silver_container = "silver" 
file_name_silver = "supermarket_sales_cleaned.parquet"
df_silver = f"abfss://{silver_container}@{storage_account}.dfs.core.windows.net/{file_name_silver}"

# Write df_clean to Silver
df_clean.write.mode("overwrite").parquet(df_silver)

In [ ]:
# Show the first 20 rows
df_silver.show(20)

# Showing schema to confirm date column type
df_silver.printSchema()

# Show table view
display(df_silver)